In [1]:
## Data Initializing

In [2]:
import pandas as pd
import numpy as np
import data_handler as dh
import transport_graph as tg
import model
import time

In [2]:
net_name = 'SiouxFalls_net.tntp'
trips_name = 'SiouxFalls_trips.tntp'

In [3]:
handler = dh.DataHandler()
graph_data = handler.GetGraphData(net_name, columns_order = np.array([0, 1, 2, 4]))
graph_data['graph_table']

,Init node,Term node,Capacity,Free Flow Time
0,1,2,25900.201172,6.0
1,1,3,23403.472656,4.0
2,2,1,25900.201172,6.0
3,2,6,4958.181152,5.0
4,3,1,23403.472656,4.0
5,3,4,17110.523438,4.0
6,3,12,23403.472656,4.0
7,4,3,17110.523438,4.0
8,4,5,17782.794922,2.0
9,4,11,4908.826660,6.0


In [4]:
graph_correspondences, total_od_flow = handler.GetGraphCorrespondences(trips_name)
total_od_flow

360600.0

In [5]:
correspondences_matrix = []
for source_values in graph_correspondences.values():
    correspondences_matrix.append(list(source_values.values()))
correspondences_matrix = np.array(correspondences_matrix)
np.shape(correspondences_matrix)

(24, 24)

In [6]:
graph = tg.TransportGraph(graph_data)
gammas = [1.0]
flows = []
times = []
iter_nums = []

for gamma in gammas:
    tic = time.time()
    print('gamma = ' + str(gamma))
    result = model.model_solve(graph, graph_correspondences, total_od_flow, 
                           gamma = gamma, mu = 0.25, rho = 0.15, epsilon = 1e-3, max_iter = 5000, verbose = True)
    toc = time.time()
    print('Elapsed time: {:.0f} sec'.format(toc - tic))
    flows.append(result['flows'])
    times.append(result['times'])
    iter_nums.append(result['iter_num'])

gamma = 1.0
Oracles created...
Universal similar triangles function...
Iterations number: 1
Duality_gap / Duality_gap_init = 1.0
Duality_gap = 11686542.6018
Iterations number: 2
Duality_gap / Duality_gap_init = 0.882095305256
Duality_gap = 10308644.3637
Iterations number: 3
Duality_gap / Duality_gap_init = 0.68942150899
Duality_gap = 8056953.83541
Iterations number: 4
Duality_gap / Duality_gap_init = 0.440754322554
Duality_gap = 5150894.16746
Iterations number: 5
Duality_gap / Duality_gap_init = 0.225617251498
Duality_gap = 2636685.62133
Iterations number: 6
Duality_gap / Duality_gap_init = 0.117957172127
Duality_gap = 1378511.51725
Iterations number: 7
Duality_gap / Duality_gap_init = 0.0691779204099
Duality_gap = 808450.713975
Iterations number: 8
Duality_gap / Duality_gap_init = 0.0399727596292
Duality_gap = 467143.358319
Iterations number: 9
Duality_gap / Duality_gap_init = 0.0199301329379
Duality_gap = 232914.347639
Iterations number: 10
Duality_gap / Duality_gap_init = 0.01085871

In [7]:
def ReadAnswer(filename):
    with open(filename) as myfile:
        lines = myfile.readlines()
    lines = np.array(lines)[range(1, len(lines))]
    values_dict = {'flow': [], 'time': []}
    for line in lines:
        line = line.strip('[ \n]')
        nums = line.split(' \t')
        values_dict['flow'].append(float(nums[2]))
        values_dict['time'].append(float(nums[3]))
    return values_dict

In [8]:
values_dict = ReadAnswer('SiouxFalls_flow.tntp')
flow_star = values_dict['flow']
time_star = values_dict['time']
delta = []
for flow in flows:
    delta.append(np.linalg.norm(flow - flow_star, ord = 1) / np.linalg.norm(flow_star, ord = 1))
delta

[0.034394390274592956]

### DATA FOR CORRESPONDENCES CALCULATIONS:

Матрица корреспонденций, к которой должен быть близок ответ вашего метода:

In [9]:
correspondences_matrix
np.shape(correspondences_matrix)

(24, 24)

Входные данные для вашего метода:

Средние временные издержки

In [14]:
times_average = result['time_average_w']
np.shape(times_average)

(24, 24)

Количество выезжающих из каждого района:

In [11]:
L_array = np.sum(correspondences_matrix, axis = 1)

Количество въезжающих в каждый район:

In [12]:
W_array = np.sum(correspondences_matrix, axis = 0)